In [1]:
LANGUAGE_MODEL_NAME = "phi3:14b"
SPACY_PIPELINE = "en_core_web_sm"
TARGET_SENTENCE_COUNT = 100
OUTPUT_FILE_COUNT = 10
OUTPUT_DIRECTORY = "out"
OLLAMA_BASE_URL = "http://host.docker.internal:11434"

In [2]:
from langchain_community.llms.ollama import Ollama

llm = Ollama(model=LANGUAGE_MODEL_NAME, base_url=OLLAMA_BASE_URL)

llm.invoke("Hi.")

'Hello! How can I assist you today?'

In [3]:
for chunk in llm.stream("Generate a 100-sentence fictitious article."):
    print(chunk, flush=True, end="")

In the quaint town of Serendipity Springs, nestled in the heartland of America, lies an enchanting secret that has been passed down through generations. This hidden gem is known to few and holds within its boundaries a magical realm where anything becomes possible. A place beyond the reaches of ordinary life, it's here that dreamers find solace, artists unlock their creativity, and adventurers discover new worlds.

Serendipity Springs was founded over 200 years ago by pioneering families who sought to create a sanctuary from the hardships of frontier life. Their legacy lives on in the charming architecture that lines the streets of the town square, where history is etched into every brick and mortar.

The magic of Serendipity Springs begins at dawn when the sun's golden rays first touch the roohrenburg fountain at the center of the square. As it happens each morning, an ethereal glow envelops the town, casting a spell that awakens its residents from slumber and fills their hearts with 

In [4]:
import spacy

spacy.cli.download(SPACY_PIPELINE)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 17.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.llms.ollama import Ollama
from spacy.language import Language
from spacy.tokens.doc import Doc

streaming_handler = StreamingStdOutCallbackHandler()


def generate_article(target_sentence_count: int) -> str:
    llm = Ollama(
        model=LANGUAGE_MODEL_NAME,
        base_url=OLLAMA_BASE_URL,
        callbacks=[streaming_handler],
    )
    memory = ConversationBufferMemory()
    conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

    first_prompt_template = PromptTemplate(
        template=(
            "Create an original, complete piece of article that is at least {target_sentence_count} sentences long. "
            "Do not summarize or abbreviate any part; continue writing until you've reached at least {target_sentence_count} sentences while maintaining coherence and relevance throughout."
        ),
        input_variables=["target_sentence_count"],
    )
    continuation_prompt_template = PromptTemplate(
        template="Write a sequel to the previous article.",
        input_variables=["target_sentence_count"],
    )

    result: str = conversation.predict(
        input=first_prompt_template.format(
            target_sentence_count=target_sentence_count,
        )
    )

    nlp: Language = spacy.load(name=SPACY_PIPELINE)
    doc: Doc = nlp(result)
    sentence_count = len(list(doc.sents))
    print(
        f"---\n⌛ Current sentence count: {sentence_count} / {target_sentence_count}\n---"
    )

    while sentence_count < target_sentence_count:
        response: str = conversation.predict(
            input=continuation_prompt_template.format(
                target_sentence_count=target_sentence_count,
            )
        )

        result = "\n\n".join(
            (
                result,
                response,
            )
        )
        doc = nlp(result)
        sentence_count = len(tuple(doc.sents))
        print(
            f"---\n⌛ Current sentence count: {sentence_count} / {target_sentence_count}\n---"
        )

    return result

In [6]:
from datetime import datetime
from pathlib import Path

parent_dir_path: Path = Path("__file__").parent
output_dir_path: Path = parent_dir_path.joinpath(OUTPUT_DIRECTORY)
output_dir_path.mkdir(parents=True, exist_ok=True)

for i in range(OUTPUT_FILE_COUNT):
    print("---\n📝 Generating new article... Please wait for a while.\n---")
    result: str = generate_article(target_sentence_count=TARGET_SENTENCE_COUNT)

    current_time: str = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    filename: str = f"article_{current_time}.txt"
    filepath: Path = output_dir_path.joinpath(filename)
    with open(file=filepath, mode="w", encoding="UTF8") as f:
        f.write(result)
    print(f"---\n✨ Generated article saved to:\n{filepath.resolve()}\n---")

---
📝 Generating new article... Please wait for a while.
---


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Create an original, complete piece of article that is at least 100 sentences long. Do not summarize or abbreviate any part; continue writing until you've reached at least 100 sentences while maintaining coherence and relevance throughout.
AI:
Title: The Enigmatic World of Bioluminescent Organisms

Introduction:

In the vast expanse of our planet, a mesmerizing phenomenon has captivated researchers, scientists, and nature enthusiasts alike - bioluminescence. This natural spectacle occurs when living organisms produce their own light through chemical reactions within their bodies. Bioluminescent creatur